In [1]:
PART_1_SKIP = True
PART_2_SKIP = True
PART_3_SKIP = True

In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import skimage
from PIL import Image
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.util import crop, pad
from skimage.morphology import label
from skimage.color import rgb2gray, gray2rgb

import os

import zipfile
z = zipfile.ZipFile('Dancer_Images.zip', 'w')

In [3]:
cap = cv2.VideoCapture('/home/aman/Downloads/dancing_silhouette.mp4')
print(cap.get(cv2.CAP_PROP_FPS))

29.97003017816262


In [4]:
%%time

if PART_1_SKIP == False:
    try:
        if not os.path.exists('data'):
            os.makedirs('data')
    except OSError:
        print ('Error: Creating directory of data')

    currentFrame = 0
    count = 0
    TRAIN_SIZE = 27000
    FRAME_SKIP = 2
    IMG_WIDTH = 96
    IMG_HEIGHT = 64
    IMG_CHANNELS = 1
    X_train = np.zeros((TRAIN_SIZE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype='float32')
    
    video = cv2.VideoWriter('Simple_Shadow_Dancer_Video.avi',cv2.VideoWriter_fourcc(*"MJPG"), 30, (IMG_WIDTH, IMG_HEIGHT), False)

    while(count < TRAIN_SIZE):
        try:
            ret, frame = cap.read()

            if currentFrame % FRAME_SKIP == 0:
                count += 1
                if count % int(TRAIN_SIZE/10) == 0:
                    print(str((count/TRAIN_SIZE)*100)+"% done")
                # preprocess frames
                img = frame
                img = rgb2gray(img)
                img = resize(img, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
                img[img > 0.2] = 255
                img[img <= 0.2] = 0
                # save frame to zip and new video sample
                name = './data/frame' + str(count) + '.jpg'
                cv2.imwrite(name, img)
                #video.write(gray2rgb(img.astype('uint8')))
                video.write(img.astype('uint8'))
                z.write(name)
                os.remove(name)
                # save image to training set if training directly to part 2
                img = img.astype('float32') / 255.
                X_train[count] = img
        except:
            print('Frame error')
            break
        currentFrame += 1

    print(str(count)+" Frames collected")
    cap.release()
    z.close()
    video.release()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.29 µs


In [7]:
import sys
import random
import warnings
from pylab import imshow, show, get_cmap
from tqdm import tqdm
from itertools import chain

from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, UpSampling2D, Flatten, Reshape
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from keras import backend as k
import tensorflow as tf

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed


In [ ]:
if PART_1_SKIP:
    IMG_WIDTH = 96
    IMG_HEIGHT = 64
    IMG_CHANNELS = 1
    INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    TRAIN_PATH = 'data/'
    train_ids = next(os.walk(TRAIN_PATH))[2]
    #l = list(os.listdir(TRAIN_PATH))
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS ), dtype = 'float32')
    missing_count = 0
    print('Getting training images...')
    
    for n, filename in enumerate(tqdm(os.listdir(TRAIN_PATH))):
        PATH = TRAIN_PATH + filename
        try:
            img = imread(PATH)
            img = img.astype('float32') / 255
            img = resize(img, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
            X_train[n-missing_count] = img
        except:
            print('Problem with %s', PATH)
            missing_count += 1
    print(f'Total missing {missing_count}')
    
else:
    INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    

  1%|          | 156/27000 [00:00<00:17, 1558.58it/s]

Getting training images...


 81%|████████▏ | 21992/27000 [00:13<00:03, 1567.57it/s]